In [1]:
import os
from os.path import join as pjoin
import time
import argparse
import random


from env.USVEnv import USVEnv
from utils.logger import Logger
from utils.load_model import load_config, load_model, save_config, save_model 
from env.heuristic.policy_red import PigeonRed
from env.heuristic.policy_blue import * #DDPG, MADDPG, HeuristicBluePolicy, SimpleActorCritic, GNNModel, GNNEmbeddingLayer, GNNCommunicationLayer, MLP, MLP_Q
import torch

In [8]:
from utils.w_comm_config import Config
model_dir = r'C:\Users\rvmzw\Documents\GitHub\USV_Swarm_MARL\results_for_analysis\old_w_comm_team_724_model_49999.pth'



config = Config()
config.num_red = 10
config.num_blue = 6
config.communication_range = 7.5
config.random_seed = 9723

In [9]:
env = USVEnv(config)
obs_b, obs_r = env.reset()

agent_red = PigeonRed(config)
agent_blue = MADDPG(config)

load_model(agent_blue, model_dir)


c:\Users\rvmzw\Documents\GitHub\USV_Swarm_MARL\utils\load_model.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file_path)


In [10]:
# Adjust for evaluation
agent_blue.noise = 0
agent_blue.num_blue = config.num_blue
agent_blue.num_red = config.num_red
agent_blue.actor_gnn.conv3.communication_range = config.communication_range


In [11]:
print(config)

In [12]:
cnt = 0 
num_episode = 500
config.exp_name = "exp_2_approachB"

root = pjoin("results", config.exp_name)
os.makedirs(root, exist_ok=True)
logger = Logger(filename=pjoin(root, 'exp_2_approachB.log'), config=config, mode='eval')

while True:
    with torch.no_grad():  # Make a decision
        action_b, curr_graph = agent_blue.get_action(obs_b)
        action_r = agent_red.get_action(obs_r)

    # Environment step
    (next_obs_b, next_obs_r), (reward_b, reward_r), done, _, info = env.step(action_b, action_r)

    # Assign next state
    obs_b = next_obs_b  
    obs_r = next_obs_r


    for info_ in info:
        if info_['seed'] <= num_episode:
            logger.log_episode(info_)
            cnt += 1

    if num_episode == cnt:
            break


12-24 12:53:07 | Blue-H vs Red-H
12-24 12:53:07 | Directory: results\exp_2_approachA\evaluation.log
12-24 12:53:10 | Episode:     2 | blue win:   0.000 | red win:   1.000 | draw:   0.000 | blue reward:   9.286 | timestep:  83.000 | blue catch:   0.000 | blue attack:   1.257 | blue island dead:   1.000 | blue reward feedback:   0.000 | seed:   4.000
12-24 12:53:10 | Episode:     3 | blue win:   0.000 | red win:   1.000 | draw:   0.000 | blue reward:   9.246 | timestep:  83.000 | blue catch:   0.000 | blue attack:   1.230 | blue island dead:   1.000 | blue reward feedback:   0.000 | seed:   5.000
12-24 12:53:10 | Episode:     4 | blue win:   0.000 | red win:   1.000 | draw:   0.000 | blue reward:  19.168 | timestep:  83.000 | blue catch:   1.000 | blue attack:   2.179 | blue island dead:   0.000 | blue reward feedback:   0.000 | seed:   7.000
12-24 12:53:10 | Episode:     5 | blue win:   0.000 | red win:   1.000 | draw:   0.000 | blue reward:   9.947 | timestep:  83.000 | blue catch:   0